In [22]:
import os
import pandas as pd
import numpy as np
import random
import gc
import time
from tqdm import tqdm

In [ ]:
train_df=pd.read_hdf('totalExposureLog.h5')

In [8]:
train_df.head()

,aid_request,request_time,aid_location,uid,aid,aid_size,bid,pctr,quality_ecpm,totalEcpm
0,53991770,1550409746,94,1160618,451525,50,46,47.218750,944.340027,3122.340088
1,25942318,1550370892,79,203814,214797,64,10,49.093750,981.880005,1471.880005
2,66156247,1550416600,18,808543,92253,40,96,3.824219,76.480003,443.584015
3,5935886,1550365898,198,7270,160082,64,85,6.121094,122.459999,642.914978
4,11624425,1550361159,168,852707,253902,64,60,4.328125,86.580002,346.320007


In [ ]:
#做时间转换 计算当前日期
train_df['request_day']=train_df['request_time']//(3600*24)

# 区分不同广告标准： 广告ID 请求日期

#统计一天内的曝光量（即出现次数）
tmp = pd.DataFrame(train_df.groupby(['aid','request_day'])['bid'].nunique()).reset_index()
tmp.columns=['aid','request_day','bid_unique']
train_df=train_df.merge(tmp,on=['aid','request_day'],how='left')
#总曝光次数
tmp = pd.DataFrame(train_df.groupby(['aid','request_day']).size()).reset_index()
#平均bid
tmp_1 = pd.DataFrame(train_df.groupby(['aid','request_day'])['bid'].mean()).reset_index()
tmp.columns=['aid','request_day','imp']
del train_df['bid']
tmp_1.columns=['aid','request_day','bid']
#添加特征后
####终于可以去除多余数据，终于不用看1E个数据了..计算机卡死
train_df=train_df.merge(tmp,on=['aid','request_day'],how='left')
train_df=train_df.merge(tmp_1,on=['aid','request_day'],how='left')
train_df=train_df.drop_duplicates(['aid','request_day'])
#去除出价过高和曝光过高数据
train_df=train_df[train_df['imp']<=3000]
train_df=train_df[train_df['bid']<=1000]
train_df.to_csv("train.csv")

 95%|█████████████████████████████████████████████████████████████▌   | 96950851/102386695 [03:53<00:15, 341555.00it/s]

In [ ]:
train_df.to_csv("train.csv")

In [ ]:
#时
wday=[]
hour=[]
minute=[]
for x in tqdm(train_df['request_time'].values,total=len(train_df)):
    localtime=time.localtime(x)
    wday.append(localtime[6])
    hour.append(localtime[3])
    minute.append(localtime[4])
#统计广告投放时段  （周几  几点 ） 两个重要指标
train_df['wday']=wday
train_df['hour']=hour
train_df['minute']=minute
train_df['period_id']=train_df['hour']*2+train_df['minute']//30

#选最后一天的最为测试样本
dev_df=train_df[train_df['request_day']==17974]
#及时删除不用的数据
del dev_df['period_id']
del dev_df['minute']
del dev_df['hour']
log=train_df

log=log.merge(tmp,on=['aid','request_day'],how='left')
log[log['request_day']<17974].to_csv('user_log_train.csv')
dev_df.to_csv('user_log_test.csv')
del log
del tmp
gc.collect()
del train_df['period_id']
del train_df['minute']
del train_df['hour']    

In [4]:
a = time.localtime(10000)

In [8]:
c = np.fromfile('ad_operation.dat')

In [12]:
c[1]

1.3188927118252273e-259

In [44]:
aids=[]
with open('ad_operation.dat','r') as f:
    for line in f:
        line=line.strip().split('\t')
        try:
#line[1] 中存储操作时间
            if line[1]=='20190230000000':
                line[1]='20190301000000'
# 改为时间戳模式 和其他数据相匹配
            if line[1]!='0':
                request_day=time.mktime(time.strptime(line[1], '%Y%m%d%H%M%S'))//(3600*24)
            else:
                request_day=0
        except:
            print(line[1])

        if len(aids)==0:
            aids.append([int(line[0]),0,"NaN","NaN"])
        elif aids[-1][0]!=int(line[0]):
            for i in range(max(17930,aids[-1][1]+1),17975):
                aids.append(aids[-1].copy())
                aids[-1][1]=i
            aids.append([int(line[0]),0,"NaN","NaN"])               
        elif request_day!=aids[-1][1]:
            for i in range(max(17930,aids[-1][1]+1),int(request_day)):
                aids.append(aids[-1].copy())
                aids[-1][1]=i                
            aids.append(aids[-1].copy())
            aids[-1][1]=int(request_day)
        if line[3]=='3':
            aids[-1][2]=line[4]
        if line[3]=='4':
            aids[-1][3]=line[4]
ad_df=pd.DataFrame(aids)
# 人群定向 投放时段
ad_df.columns=['aid','request_day','crowd_direction','delivery_periods']

In [45]:
ad_df

,aid,request_day,crowd_direction,delivery_periods
0,593323,0,all,"281474976710655,281474976710655,28147497671065..."
1,593323,17930,all,"281474976710655,281474976710655,28147497671065..."
2,593323,17931,all,"281474976710655,281474976710655,28147497671065..."
3,593323,17932,all,"281474976710655,281474976710655,28147497671065..."
4,593323,17933,all,"281474976710655,281474976710655,28147497671065..."
...,...,...,...,...
1793880,29322,17971,NaN,NaN
1793881,29322,17972,NaN,NaN
1793882,29322,17973,NaN,NaN
1793883,29322,17974,NaN,NaN


数据

line[0]广告id   line[1]操作时间  line[2]定向人群  line[3]时段

In [46]:
#清洗掉未进行操作的广告数据
ad_df=ad_df.drop_duplicates(['aid','request_day'],keep='last')
ad_df['request_day']+=1

ad_df['is']=ad_df['crowd_direction'].apply(lambda x:type(x)==str)
ad_df=ad_df[ad_df['is']==True]
ad_df=ad_df[ad_df['crowd_direction']!="NaN"]
ad_df=ad_df[ad_df['delivery_periods']!="NaN"]

D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [47]:
#去重复 合并两特征集
ad_df=ad_df.drop_duplicates(['aid','request_day'],keep='last')
ad_df['request_day']+=1
train_df=train_df.merge(ad_df,on=['aid','request_day'],how='left')

,aid,request_day,crowd_direction,delivery_periods,is
0,593323,1,all,"281474976710655,281474976710655,28147497671065...",True
1,593323,17931,all,"281474976710655,281474976710655,28147497671065...",True
2,593323,17932,all,"281474976710655,281474976710655,28147497671065...",True
3,593323,17933,all,"281474976710655,281474976710655,28147497671065...",True
4,593323,17934,all,"281474976710655,281474976710655,28147497671065...",True
...,...,...,...,...,...
1781965,300864,17971,"age:202,394|area:12960","281474976710655,281474976710655,28147497671065...",True
1781966,300864,17972,"age:202,394|area:12960","281474976710655,281474976710655,28147497671065...",True
1781967,300864,17973,"age:202,394|area:12960","281474976710655,281474976710655,28147497671065...",True
1781968,300864,17974,"age:202,394|area:12960","281474976710655,281474976710655,28147497671065...",True


In [ ]:
aids=set()
exit_aids=set()
with open('ad_operation.dat','r') as f:
    for line in f:
        line=line.strip().split('\t')
        if line[1]=='20190230000000':
            line[1]='20190301000000'
        if line[1]!='0':
            request_day=time.mktime(time.strptime(line[1], '%Y%m%d%H%M%S'))//(3600*24)
        else:
            request_day=0
#选取一天作为验证集
        if request_day==17974:
            aids.add(int(line[0]))
        exit_aids.add(int(line[0]))
dev_df = pd.read_csv('user_log_test.csv')
#找到对应天的广告
dev_df['is']=dev_df['aid'].apply(lambda x: x in aids)
dev_df=dev_df[dev_df['is']==False]
#去除重复
dev_df['is']=dev_df['aid'].apply(lambda x: x in exit_aids)
dev_df=dev_df[dev_df['is']==True]
#过滤当天出价不唯一的广告
tmp = pd.DataFrame(dev_df.groupby('aid')['bid'].nunique()).reset_index()
tmp.columns=['aid','bid_unique']
dev_df=dev_df.merge(tmp,on='aid',how='left')
dev_df=dev_df[dev_df['bid_unique']==1]

In [ ]:
#验证集 清洗数据
tmp = pd.DataFrame(dev_df.groupby('aid')['bid'].nunique()).reset_index()
tmp.columns=['aid','bid_unique']
dev_df=dev_df.merge(tmp,on='aid',how='left')
dev_df=dev_df[dev_df['bid_unique']==1]
#统计广告当天的曝光量
tmp = pd.DataFrame(dev_df.groupby('aid').size()).reset_index()
tmp.columns=['aid','imp']
dev_df=dev_df.merge(tmp,on='aid',how='left')
dev_df=dev_df.drop_duplicates('aid')
#过滤未出现在广告操作文件的广告
ad_df=extract_setting()
ad_df=ad_df.drop_duplicates(['aid'],keep='last')
dev_df=dev_df.merge(ad_df,on='aid',how='left')
dev_df=dev_df[dev_df['crowd_direction']!="NaN"]
dev_df=dev_df[dev_df['delivery_periods']!="NaN"].reset_index()